In [ ]:
import  pandas as pd

In [ ]:
filename = 'datapikabu_dataset_new.csv'
filename_processed = 'datapikabu_dataset_new_processed.csv'

In [ ]:
df = pd.read_csv(filename)
df.head()

In [ ]:
import pandas as pd
import string
import re
import nltk
import collections

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import pymorphy2


stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def preprocessing(text):
    text = str(text)
    text = re.sub(r"^\s+|\n|\r|\t|\s+$", "", text) # отступы
    text = re.sub(r'.[0-9]+', ' ', text) # цифры
    text = re.sub(r'.[a-z]+', ' ', text) # английские символы
    text = re.sub(r'[^\w\s]',' ', text) # пунктуация
    text = text.lower() # регистр
    text = " ".join([word for word in text.split(" ") if (word not in stop_words)]) # стоп-слова
    text = " ".join([t for t in text.split(" ") if len(t) > 0]) # лишние пробелы
    text = " ".join([morph.parse(word)[0].normal_form for word in text.split(" ")]) # лемматизация
    return str(text)

In [ ]:
df.index

In [ ]:
df_enough = df[df['Text'].map(lambda x: len(str(x)) > 1000)]

In [ ]:
new_filename = "pikabu_dataset_longText.csv"

In [ ]:
df_enough.to_csv(new_filename)

### В лоб на 2 дня

In [ ]:
from tqdm import tqdm 

df = pd.read_csv(filename)
df = df[df['Text'].map(lambda x: len(str(x)) > 1000)]

for index, row in tqdm(df.iterrows()):
    df.loc[index, 'Text'] = preprocessing(df.loc[index, 'Text'])
    df.loc[index, 'Tags'] = preprocessing(df.loc[index, 'Tags'])
    df.loc[index, 'Title'] = preprocessing(df.loc[index, 'Title'])

df.to_csv("pikabu_dataset_processed.csv")

In [ ]:
df.columns

### with Spark

In [ ]:
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType

In [ ]:
from functools import reduce
from pyspark.sql.functions import col

In [ ]:
os.environ["PYSPARK_PYTHON"] = "python3"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

In [ ]:
df_spark = spark.read.csv(new_filename, inferSchema=True, header=True).toDF('Number', 'Number2', 'FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text')

In [ ]:
df_spark.show()

In [ ]:
filename_processed = "pikabu_dataset_processed.csv"

In [ ]:
from pyspark.sql.types import StringType

filename = "hot_dataset.csv"

spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

df_spark = spark.read.csv(filename, inferSchema=True, header=True).toDF('Number', 'FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text')

prepr = F.udf(preprocessing, StringType())

df_spark.withColumn('Text2', prepr(df_spark['Text'])) \
.toPandas().to_csv("hot_dataset_processed.csv")

In [ ]:
df_spark.withColumn('Text', reduce(preprocessing, [col('Text')])) \
.withColumn('Title', reduce(preprocessing, [col('Title')])) \
.withColumn('Tags', reduce(preprocessing, [col('Tags')])) \
.toPandas().to_csv(filename_processed)

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession

filename = "hot_dataset.csv"
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

df_spark = spark.read.csv(filename, inferSchema=True, header=True).toDF('Number', 'FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text')

ss = SparkSession.builder

sparkSession = ss.getOrCreate()
sc = sparkSession.sparkContext
sqlContext = SQLContext(sc)

def rowwise_function(row):
    row_dict = row.asDict()
    row_dict['Text'] = preprocessing(row_dict['Text'])
    row_dict['Tags'] = preprocessing(row_dict['Tags'])
    row_dict['Title'] = preprocessing(row_dict['Title'])
    newrow = Row(**row_dict)
    return newrow


dfSchema = StructType([
    StructField('Number',IntegerType(), True), 
    StructField('FileName', StringType(), True), 
    StructField('Title', StringType(),True), 
    StructField('Link', StringType(), True),
    StructField('ArticleId', IntegerType(), True), 
    StructField('Date', StringType(), True), 
    StructField('Views', IntegerType(), True),
    StructField('Author', StringType(), True), 
    StructField('Tags', StringType(), True), 
    StructField('AmountComments', IntegerType(), True), 
    StructField('Rating', IntegerType(), True), 
    StructField('Text', StringType(), True)
        ])

spark_rdd = df_spark.rdd
spark_rdd_new = spark_rdd.map(lambda row: rowwise_function(row))

spark_rdd_new = sqlContext.createDataFrame(spark_rdd_new dfSchema)
spark_rdd_new.show()


In [ ]:
spark_rdd_new = sqlContext.createDataFrame(spark_rdd_new, dfSchema)

In [ ]:
spark_rdd_new.toPandas().to_csv("hot_dataset_processed.csv")

In [ ]:
spark_rdd_new.write.format('com.databricks.spark.csv').save('mycsv.csv')

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession

filename = "hot_dataset.csv"

spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

df_spark = spark.read.csv(filename, inferSchema=True, header=True).toDF('Number', 'FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text')

ss = SparkSession.builder
sparkSession = ss.getOrCreate()
sc = sparkSession.sparkContext
sqlContext = SQLContext(sc)

def rowwise_function(row):
    row_dict = row.asDict()
    row_dict['Text'] = preprocessing(row_dict['Text'])
    row_dict['Tags'] = preprocessing(row_dict['Tags'])
    row_dict['Title'] = preprocessing(row_dict['Title'])
    newrow = Row(**row_dict)
    return newrow


dfSchema = StructType([
    StructField('Number',IntegerType(), True), 
    StructField('FileName', StringType(), True), 
    StructField('Title', StringType(),True), 
    StructField('Link', StringType(), True),
    StructField('ArticleId', StringType(), IntegerType(), True), 
    StructField('Date', StringType(), True), 
    StructField('Views', IntegerType(), True),
    StructField('Author', StringType(), True), 
    StructField('Tags', StringType(), True), 
    StructField('AmountComments', IntegerType(), True), 
    StructField('Rating', IntegerType(), True), 
    StructField('Text', StringType(), True)
        ])

spark_rdd = df_spark.rdd
spark_rdd_new = spark_rdd.map(lambda row: rowwise_function(row))

spark_rdd_new = sqlContext.createDataFrame(spark_rdd_new, dfSchema)
spark_rdd_new.show()
#spark_rdd_new.toPandas().to_csv("hot_dataset_processed.csv")

In [ ]:
name = "hot_dataset_processed.csv"
spark_rdd_new.repartition(1).write.csv(name, sep=',')

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession

filename = "hot_dataset.csv"

spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

df_spark = spark.read.csv(filename, inferSchema=True, header=True).toDF('Number', 'FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text')

ss = SparkSession.builder
sparkSession = ss.getOrCreate()
sc = sparkSession.sparkContext
sqlContext = SQLContext(sc)

def rowwise_function(row):
    row_dict = row.asDict()
    row_dict['Text'] = preprocessing(row_dict['Text'])
    row_dict['Tags'] = preprocessing(row_dict['Tags'])
    row_dict['Title'] = preprocessing(row_dict['Title'])
    newrow = Row(**row_dict)
    return newrow


dfSchema = StructType([
    StructField('Number',IntegerType(), True), 
    StructField('FileName', StringType(), True), 
    StructField('Title', StringType(),True), 
    StructField('Link', StringType(), True),
    StructField('ArticleId', IntegerType(), True), 
    StructField('Date', StringType(), True), 
    StructField('Views', IntegerType(), True),
    StructField('Author', StringType(), True), 
    StructField('Tags', StringType(), True), 
    StructField('AmountComments', IntegerType(), True), 
    StructField('Rating', IntegerType(), True), 
    StructField('Text', StringType(), True)
        ])

spark_rdd = df_spark.rdd
spark_rdd_new = spark_rdd.map(lambda row: rowwise_function(row))

spark_rdd_new = sqlContext.createDataFrame(spark_rdd_new, dfSchema)
# spark_rdd_new.show()
spark_rdd_new.toPandas().to_csv("hot_dataset_processed.csv")

In [ ]:
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession

filename = "hot_dataset.csv"
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

df_spark = spark.read.csv(filename, inferSchema=True, header=True).toDF('Number','FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text')

ss = SparkSession.builder
sparkSession = ss.getOrCreate()
sc = sparkSession.sparkContext
sqlContext = SQLContext(sc)

def rowwise_function(row):
    row_dict = row.asDict()
    row_dict['Text'] = preprocessing(row_dict['Text'])
    row_dict['Tags'] = preprocessing(row_dict['Tags'])
    row_dict['Title'] = preprocessing(row_dict['Title'])
    newrow = Row(**row_dict)
    return newrow

spark_rdd = df_spark.rdd
spark_rdd_new = spark_rdd.map(lambda row: rowwise_function(row))

In [ ]:
spark_rdd_new = sqlContext.createDataFrame(spark_rdd_new, dfSchema)

In [ ]:
dfSchema = StructType([
    StructField('Number',IntegerType(), True), 
    StructField('FileName', StringType(), True), 
    StructField('Title', StringType(),True), 
    StructField('Link', StringType(), True),
    StructField('ArticleId', IntegerType(), True), 
    StructField('Date', StringType(), True), 
    StructField('Views', IntegerType(), True),
    StructField('Author', StringType(), True), 
    StructField('Tags', StringType(), True), 
    StructField('AmountComments', IntegerType(), True), 
    StructField('Rating', IntegerType(), True), 
    StructField('Text', StringType(), True)
        ])

spark_rdd_new = sqlContext.createDataFrame(spark_rdd_new, dfSchema)

In [ ]:
spark_rdd_new.show()

In [ ]:
spark_rdd_new.toPandas().to_csv("hot_dataset_processed.csv")

In [ ]:
spark_rdd_new.toPandas().to_csv(filename_processed)

### Прочее

In [ ]:
spark.read.option("escape","\"")
df_spark = spark.read.format("csv").option("header", "true").load(filename)

In [ ]:
frame = pd.read_csv(filename, encoding='utf-8') # Prod
#frame = frame.where(pd.notnull(frame), None) # Fill NaN with Null


dfSchema = StructType([
    StructField('Number',IntegerType(), True), 
    StructField('FileName', StringType(), True), 
    StructField('Title', StringType(),True), 
    StructField('Link', StringType(), True),
    StructField('ArticleId', StringType(), IntegerType(), True), 
    StructField('Date', StringType(), True), 
    StructField('Views', IntegerType(), True),
    StructField('Author', StringType(), True), 
    StructField('Tags', StringType(), True), 
    StructField('AmountComments', IntegerType(), True), 
    StructField('Rating', IntegerType(), True), 
    StructField('Text', StringType(), True)
        ])

df_s = spark.createDataFrame(frame, dfSchema)